In [ ]:
#My goal
""" 
From my findingss the difference between an exceptional algorithm and a subpar algorithm is the difference between the two graphs below
If we can find a way to smooth out the micro-flips, we can get a much better P&L curve. 
Feel free to play around with the hyperparameters too but they have already been tuned and backtested to ensure minimal overfitting.

TLDR:
Your task if you choose to take it, see what you can do with adding features, stacking models, or otherwise improving the algorithm below to try get a match on the "answer" graph. 
Keep in mind the graphs showed below are an example snapshot feel free to change T-ranges and intrument.

I reckon if we can nail this down we are as good as won.
"""

#All hyperparameters are explained below:
'''
------------------------------------------------------------------------------------------------------------------------------------------------------------------
HMA MODEL:

hma_period (int)
The look-back length for the HMA. Larger → smoother, slower to react; smaller → more responsive but choppier.

N_trend (int)
“Strong-trend” bar count. Once you’ve been in one direction for ≥ N bars, the buffered signal will flip immediately on the first opposite raw tick.

X_confirm (int)
“Choppy-regime” confirmation count. If you haven’t yet hit N_trend, you require X_confirm consecutive opposite raw ticks before switching.

------------------------------------------------------------------------------------------------------------------------------------------------------------------

KALMAN FILTER:

R (float)
Measurement noise variance. Larger R → filter trusts new price observations less (more smoothing).

Q_level (float)
Process noise variance for the level component. Larger→ allows the estimated level to adapt faster to changes.

Q_trend (float)
Process noise variance for the trend component. Larger→ allows the slope estimate to adapt more quickly.

pct_thresh (float)
A “dead-zone” on raw price returns. When the fractional change is below ±pct_thresh, the Kalman signal is forced to 0 (neutral), filtering out micro-noise.

------------------------------------------------------------------------------------------------------------------------------------------------------------------

PLOTTING WINDOW AND GRAPHING:

T1, T2 (int)
Start/end indices of the time axis you actually draw. Anything outside [T1,T2) is clipped.

divider_w (float)
Line-width of each thin vertical bar line separating one timestep from the next.

divider_a (float)
Alpha/transparency of those divider lines (so they don’t overwhelm the chart).
'''



In [ ]:
#Regular algorithm I made using a combination of HMA and Kalman filters and tuned relevant paramters. 

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Load prices_file directly from the current working directory."""
    f = Path(prices_file)
    if not f.exists():
        raise FileNotFoundError(f"Cannot find {prices_file} in {Path.cwd()}")
    df = pd.read_csv(f, sep=r"\s+", header=None)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
    • If current side has lasted ≥N bars, flip on first opposite bar.
    • Else need X consecutive opposite bars to flip.
    """
    out           = np.empty_like(raw)
    state         = raw[0]
    same_streak   = 1
    opp_counter   = 0
    out[0]        = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter  = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state       = r
                same_streak = 1
                opp_counter = 0
            else:
                same_streak = 0
        out[t] = state
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 8,
    N_trend        : int   = 6,
    X_confirm      : int   = 3,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    pct_thresh     : float = 0.001,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.3,
    divider_a      : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) Buffered‑HMA signal
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman signal
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int)
    k_dir[1:]  = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement + persistence
    agree      = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state      = agree.copy()
    for t in range(1, n):
        if state[t] == 0:
            state[t] = state[t-1]

    # 4) Slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    h   = h_series[T1:T2]
    k   = k_series[T1:T2]
    s   = state[T1:T2]

    # 5) Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shade regimes
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi, facecolor="green" if cur == 1 else "red", alpha=0.22)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1, facecolor="green" if cur == 1 else "red", alpha=0.22)

    ax.set_title(
        f"Instr {instrument}: persistent regime when models disagree\n"
        f"HMA N={N_trend}, X={X_confirm} | Kalman neutral band ±{pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout()
    plt.show()

# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  = 19,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 240,
        T2          = 440,
    )



In [ ]:
#New paramter: M_cheat
# That little post-processing pass will remove any spurts of length 1 or 2, and force the chart to stay in whichever regime was in effect just before the tiny flip. You can tune M=3, 4 or whatever feels right
# Because this uses the future (it needs to know how long the run would have been), it’s technically a look-ahead “cheat,” but it’ll let you test your hypothesis about whether those micro-flips are really dragging your P&L down.


# It still computes everything causally: HMA, Kalman, buffering, agreement, next-bar execution.
# After you’ve built the full binary regime state[t] for every bar, it walks forward and says “if this bullish (or bearish) stretch lasts less than M_cheat bars, overwrite it with the surrounding regime.”
# That necessarily uses bars beyond the current one to decide what the “true” regime should’ve been, effectively knowing the future.

# We wipe out all the little 1–3-bar whipsaws (micro-flips) that in real time you wouldn’t know were going to be short-lived.


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Load prices_file from the current working directory and return the chosen column."""
    f = Path(prices_file)
    if not f.exists():
        raise FileNotFoundError(f"Cannot find {prices_file} in {Path.cwd()}")
    df = pd.read_csv(f, sep=r"\s+", header=None)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period+1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period-1, n):
        out[i] = (w * series[i-period+1:i+1]).sum() / S
    out[:period-1] = series[:period-1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period//2), max(1, int(np.sqrt(period)))
    return wma(2*wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5) -> np.ndarray:
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1,1],[0,1]])
    H   = np.array([[1,0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0,0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + K.flatten()*y
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
      • If current side has lasted ≥N bars, flip on first opposite bar.
      • Else need X consecutive opposite bars before flipping.
    """
    out         = np.empty_like(raw)
    state       = raw[0]
    same_streak = 1
    opp_count   = 0
    out[0]      = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_count   = 0
        else:
            opp_count += 1
            if same_streak >= N or opp_count >= X:
                state       = r
                same_streak = 1
                opp_count   = 0
            else:
                same_streak = 0
        out[t] = state
    return out

def remove_short_runs(state: np.ndarray, M: int) -> np.ndarray:
    """
    Look-ahead “cheat”: any run of identical state values shorter than M
    gets merged into the prior regime (or next if at start).
    """
    out = state.copy()
    n   = len(state)
    i = 0
    while i < n:
        # find end j of this run
        j = i + 1
        while j < n and state[j] == state[i]:
            j += 1
        run_len = j - i
        if run_len < M:
            fill = out[i-1] if i>0 else (state[j] if j<n else state[i])
            out[i:j] = fill
        i = j
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree_cheat(
    instrument   : int,
    prices_file  : str   = "prices.txt",
    # HMA params
    hma_period   : int   = 8,
    N_trend      : int   = 6,
    X_confirm    : int   = 3,
    # Kalman params
    R            : float = 0.075,
    Q_level      : float = 4e-3,
    Q_trend      : float = 1e-5,
    pct_thresh   : float = 0.001,
    # minimum run-length to enforce
    M_cheat      : int   = 3,
    # window
    T1           : int   = 340,
    T2           : int   = 800,
    divider_w    : float = 0.3,
    divider_a    : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) HMA + buffered signals
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman + neutral band
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int); k_dir[1:] = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement + hold-last
    agree = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state = agree.copy()
    for t in range(1, n):
        if state[t] == 0:
            state[t] = state[t-1]

    # 4) Cheating look-ahead: remove short runs < M_cheat
    state = remove_short_runs(state, M_cheat)

    # 5) Plot slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    h   = h_series[T1:T2]
    k   = k_series[T1:T2]
    s   = state[T1:T2]

    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA({hma_period})", color="orange", lw=2)
    ax.plot(x, k, "-.", label="Kalman", color="green")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shaded regimes
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi,
                       facecolor="green" if cur==1 else "red",
                       alpha=0.25, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1,
               facecolor="green" if cur==1 else "red",
               alpha=0.25, edgecolor=None)

    ax.set_title(f"Instr {instrument}: cheat-filtered regimes (min run={M_cheat})\n"
                 f"HMA N={N_trend},X={X_confirm} | Kalman ±{pct_thresh*100:.2f}%")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree_cheat(
        instrument   = 19,
        prices_file  = "prices.txt",
        hma_period   = 4,
        N_trend      = 5,
        X_confirm    = 1,
        R            = 0.075,
        Q_level      = 4e-3,
        Q_trend      = 1e-5,
        pct_thresh   = 0.001,
        M_cheat      = 5,
        T1           = 240,
        T2           = 440,
    )


In [ ]:
#Diffeences between 500 score and -40

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ──────────────────────────── shared helpers ────────────────────────────
def load_prices(prices_file="prices.txt", T_cut=800) -> np.ndarray:
    for folder in (Path.cwd(), *Path.cwd().parents):
        p = folder / prices_file
        if p.exists():
            df = pd.read_csv(p, sep=r"\s+", header=None)
            return df.values.T[:, :T_cut]
    raise FileNotFoundError(prices_file)

def wma(x,n):
    w = np.arange(1, n+1, dtype=float); S=w.sum()
    out=np.empty_like(x,float); out[:n-1]=x[:n-1]
    for i in range(n-1,len(x)):
        out[i]=(w*x[i-n+1:i+1]).sum()/S
    return out
def hma(x,n): return wma(2*wma(x,max(1,n//2))-wma(x,n),max(1,int(np.sqrt(n))))
def kalman(x,R,Ql,Qt):
    F=np.array([[1,1],[0,1]]); H=np.array([[1,0]]); Q=np.diag([Ql,Qt])
    s=np.array([x[0],0.0]); P=np.eye(2); out=np.empty(len(x))
    for t in range(len(x)):
        s=F@s; P=F@P@F.T+Q
        y=x[t]-(H@s)[0]; S=(H@P@H.T)[0,0]+R; K=(P@H.T)/S
        s=s+K.flatten()*y; P=(np.eye(2)-K@H)@P; out[t]=s[0]
    return out
def buffered(raw,N,X):
    out=np.empty_like(raw,int); st,same,opp=raw[0],1,0; out[0]=st
    for i,r in enumerate(raw[1:],1):
        if r==st: same+=1; opp=0
        else:
            opp+=1
            if same>=N or opp>=X: st,same,opp=r,1,0
            else: same=0
        out[i]=st
    return out
def filt_grad(raw,k,w,gp,gn):
    out=raw.copy()
    for t in range(1,len(raw)):
        if raw[t]!=out[t-1]:
            s=max(0,t-w+1); y=k[s:t+1]
            m=np.polyfit(np.arange(len(y)),y,1)[0]/(y.mean() or 1.0)
            out[t]=out[t-1] if gn<=m<=gp else raw[t]
        else: out[t]=out[t-1]
    return out
def remove_partial(sig,M,feel):
    out=sig.copy(); i=0
    while i<len(sig):
        j=i+1
        while j<len(sig) and sig[j]==sig[i]: j+=1
        run=j-i
        if run<M and feel>0:
            k=int(run*feel)
            idx=np.unique(np.round(np.linspace(0,run-1,k)).astype(int))
            fill=out[i-1] if i else sig[i]
            for off in idx: out[i+off]=fill
        i=j
    return out

# ───────────────────────── parameters & data ─────────────────────────
prices = load_prices("prices.txt",800)
nInst,T = prices.shape
hma_p,N_tr,X_conf = 4,5,1
R,Ql,Qt=0.075,0.004,1e-5
pct_thresh=0.001; reg_w,gp,gn=5,0.001,-0.001
M_cheat,feel_bad=8,1.0
capital=10_000

# indicator precompute
H=np.vstack([hma(prices[i],hma_p) for i in range(nInst)])
dH=np.zeros_like(H); dH[:,1:]=(H[:,1:]-H[:,:-1])/H[:,:-1]
Hsig=np.vstack([buffered(np.where(dH[i]>0,1,-1),N_tr,X_conf) for i in range(nInst)])
K=np.vstack([kalman(prices[i],R,Ql,Qt) for i in range(nInst)])
Raw=np.where(H>K,1,-1)
Sig_base=np.vstack([filt_grad(Raw[i],K[i],reg_w,gp,gn) for i in range(nInst)])
Sig_cheat=np.vstack([remove_partial(Sig_base[i],M_cheat,feel_bad) for i in range(nInst)])
shares=(capital//prices)*(prices>0)
pos_base ,pos_cheat = Sig_base*shares , Sig_cheat*shares

# ───────────────────────── plotting settings ─────────────────────────
T1,T2 = 240,440                    # show only this window
idx   = np.arange(T1,T2)
N_plot=20                          # first N instruments
figsize=(16,6)                     # bigger graphs

for inst in range(min(N_plot, nInst)):
    price = prices[inst, T1:T2]
    sigC  = pos_cheat[inst, T1:T2]
    sigB  = pos_base [inst, T1:T2]

    fig,(ax1,ax2)=plt.subplots(2,1,figsize=figsize,sharex=True,
                               gridspec_kw={'hspace':0.03})
    for ax,sig,label in zip((ax1,ax2),(sigC,sigB),
                            ("CheatPartial","BaseModel")):
        # background shading
        run_state=sig[0]; start=idx[0]
        for t,x in zip(sig[1:],idx[1:]):
            if t!=run_state:
                color="green" if run_state>0 else "red"
                ax.axvspan(start,x,facecolor=color,alpha=0.25)
                run_state, start = t, x
        ax.axvspan(start, idx[-1]+1,
                   facecolor=("green" if run_state>0 else "red"), alpha=0.25)

        # thin black dividers
        ax.vlines(idx, price.min(), price.max(), colors="black", lw=0.3, alpha=0.3)

        ax.plot(idx, price, color="black", lw=1)
        ax.set_ylabel("Price")
        ax.set_title(f"Instr {inst} — {label}")

    ax2.set_xlabel("Timestep")
    plt.tight_layout()
    plt.show()
